In [2]:
# Importing modules

In [3]:
# data wrangling
import pandas as pd
import numpy as np


#visualization

import seaborn as sns
import matplotlib.pyplot as plt
#from math import pi

from sklearn.metrics.pairwise import cosine_similarity


# Loading and exploring the data 

In [4]:
path_movies = '/home/jh23/Documents/wissenschaft/data_science/projects/08_recommender_systems/data/ml-latest-small/movies.csv'
movies_df = pd.read_csv(path_movies)

path_ratings = '/home/jh23/Documents/wissenschaft/data_science/projects/08_recommender_systems/data/ml-latest-small/ratings.csv'
ratings_df = pd.read_csv(path_ratings)

path_tags = '/home/jh23/Documents/wissenschaft/data_science/projects/08_recommender_systems/data/ml-latest-small/tags.csv'
tags_df = pd.read_csv(path_tags)

path_links = '/home/jh23/Documents/wissenschaft/data_science/projects/08_recommender_systems/data/ml-latest-small/links.csv'
path_df = pd.read_csv(path_links)

# 2. Item-based recommendations

## 2.1 Popularity-based reccommendations

#### Calculates the scores for 10 movies with more than 10 ratings 

In [11]:

number_of_movies=10
min_no_ratings = 10

def movie_score_func(movies=movies_df, ratings=ratings_df, tags=tags_df, min_no_ratings=10, number_of_movies=10):
#    try:
#        number_of_movies = int(input('How many movies to recommend?  '))
#    except:
#        number_of_movies = 10
    average_rating = ratings.groupby('movieId')['rating'].mean()
    ratings_per_movie = ratings.groupby('movieId')['rating'].count()
    score_base = pd.merge(average_rating, ratings_per_movie, on='movieId').reset_index()
    score_base[['average_rating','no_ratings']] = score_base[['rating_x','rating_y']]
    score_base= score_base.drop(columns=['rating_x','rating_y'])
    score_base = score_base.loc[score_base['no_ratings']>min_no_ratings-1]
    score_dict = {'score':[], 'movieId':[]}
   # print(score_base)
    for movie in score_base['movieId'].to_list():
        #print(movie)
        pop_factor = float(1+(1/score_base.no_ratings.max())*score_base[score_base['movieId']==movie].no_ratings)
        rating = float(score_base[score_base['movieId']==movie].average_rating)
        score=pop_factor*rating
        score_dict['score'].append(score)
        score_dict['movieId'].append(movie)
       #return score_dict
    
    score_df = pd.merge(score_base, pd.DataFrame(score_dict), on='movieId').sort_values('score', ascending=False).head(number_of_movies)
    score_df = pd.merge(score_df,movies, how='left', on='movieId')
   #score_df = pd.merge(score_df,tags[['tag','movieId']], how='left', on='movieId')    
    return score_df

score_df=movie_score_func()
    

In [12]:

score_df.sort_values('no_ratings', ascending=False).head()

,movieId,average_rating,no_ratings,score,title,genres
1,356,4.164134,329,8.328267,Forrest Gump (1994),Comedy|Drama|Romance|War
0,318,4.429022,317,8.696499,"Shawshank Redemption, The (1994)",Crime|Drama
2,296,4.197068,307,8.113482,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,593,4.161290,279,7.690166,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
3,2571,4.192446,278,7.734999,"Matrix, The (1999)",Action|Sci-Fi|Thriller


#### Calculates the scores for all movies with more than 10 ratings 

In [13]:
#number_of_movies=10
min_no_ratings = 10

def all_movies_score_func(movies=movies_df, ratings=ratings_df, tags=tags_df, min_no_ratings=10):
    
    average_rating = ratings.groupby('movieId')['rating'].mean()
    ratings_per_movie = ratings.groupby('movieId')['rating'].count()
    score_base = pd.merge(average_rating, ratings_per_movie, on='movieId').reset_index()
    score_base[['average_rating','no_ratings']] = score_base[['rating_x','rating_y']]
    score_base= score_base.drop(columns=['rating_x','rating_y'])
    score_base = score_base.loc[score_base['no_ratings']>min_no_ratings-1]
    score_dict = {'score':[], 'movieId':[]}
   # print(score_base)
    for movie in score_base['movieId'].to_list():
        #print(movie)
        pop_factor = float(1+(1/score_base.no_ratings.max())*score_base[score_base['movieId']==movie].no_ratings)
        rating = float(score_base[score_base['movieId']==movie].average_rating)
        score=pop_factor*rating
        score_dict['score'].append(score)
        score_dict['movieId'].append(movie)
       #return score_dict
    
    all_score_df = pd.merge(score_base, pd.DataFrame(score_dict), on='movieId').sort_values('score', ascending=False)
    all_score_df = pd.merge(all_score_df,movies, how='left', on='movieId')
   #score_df = pd.merge(score_df,tags[['tag','movieId']], how='left', on='movieId')    
    return all_score_df

all_score_df=all_movies_score_func()
all_score_df

,movieId,average_rating,no_ratings,score,title,genres
0,318,4.429022,317,8.696499,"Shawshank Redemption, The (1994)",Crime|Drama
1,356,4.164134,329,8.328267,Forrest Gump (1994),Comedy|Drama|Romance|War
2,296,4.197068,307,8.113482,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,2571,4.192446,278,7.734999,"Matrix, The (1999)",Action|Sci-Fi|Thriller
4,593,4.161290,279,7.690166,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
...,...,...,...,...,...,...
2264,3593,1.657895,19,1.753639,Battlefield Earth (2000),Action|Sci-Fi
2265,1556,1.605263,19,1.697968,Speed 2: Cruise Control (1997),Action|Romance|Thriller
2266,3564,1.625000,12,1.684271,"Flintstones in Viva Rock Vegas, The (2000)",Children|Comedy
2267,2798,1.583333,12,1.641084,Problem Child (1990),Children|Comedy


## 2.2 Item-based collaborative filtering

In [14]:
all_movies_crosstab = pd.pivot_table(data=ratings_df, values='rating', index='userId', columns='movieId')

In [15]:
def movie_recommendations1_func(all_movies_crosstab=all_movies_crosstab, ratings_df=ratings_df):
    
    try:
        selected_movie_id = int(input('Insert the movie Id'))
    except:
        selected_movie_id = 356 # Forest Gump (id: 356) is the most popular movie (i.e. highest and most frequently rated)
    
    # How was this movie rated by users
    ratings_selected_movie = all_movies_crosstab[selected_movie_id]
    ratings_selected_movie[ratings_selected_movie>=0].dropna(inplace=True)

    # How similar (correlated) are other movies to the selected movie? 
    similarity_w_selected_movie = all_movies_crosstab.corrwith(ratings_selected_movie)
    similarity_w_selected_movie = pd.DataFrame(similarity_w_selected_movie, columns=['PearsonsR'])
    similarity_w_selected_movie.dropna(inplace=True)
    similarity_w_selected_movie.drop(selected_movie_id, inplace=True)

    similar_movies = pd.merge(similarity_w_selected_movie, all_score_df, how='inner', on='movieId')
    similar_movies = similar_movies.sort_values('PearsonsR',ascending=False)

    #Excluding terrible movies
    best_similar_movies = similar_movies.loc[similar_movies['average_rating']>=2.5].head(10)
    return best_similar_movies
    
pd.DataFrame(movie_recommendations1_func())

/home/jh23/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/jh23/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,movieId,PearsonsR,average_rating,no_ratings,score,title,genres
10,14,1.000000,3.833333,18,4.043060,Nixon (1995),Drama
139,280,1.000000,4.076923,13,4.238017,Murder in the First (1995),Drama|Thriller
106,222,0.983739,3.750000,16,3.932371,Circle of Friends (1995),Drama|Romance
349,900,0.968330,3.708333,12,3.843592,"American in Paris, An (1951)",Musical|Romance
475,1211,0.902914,3.884615,13,4.038111,"Wings of Desire (Himmel über Berlin, Der) (1987)",Drama|Fantasy|Romance
2148,103228,0.887852,3.607143,14,3.760638,Pacific Rim (2013),Action|Adventure|Sci-Fi|IMAX
194,383,0.859338,3.095238,21,3.292806,Wyatt Earp (1994),Western
1070,3019,0.825662,4.150000,10,4.276140,Drugstore Cowboy (1989),Crime|Drama
384,991,0.814381,3.318182,11,3.429124,Michael Collins (1996),Drama
23,28,0.813326,4.227273,11,4.368610,Persuasion (1995),Drama|Romance


## 2.3 User-based collaborative filtering

In [16]:
def movie_recommendations2_func(all_movies_crosstab=all_movies_crosstab):
    all_movies_crosstab0 = all_movies_crosstab.fillna(0)
    user_similarities = pd.DataFrame(cosine_similarity(all_movies_crosstab0), columns=all_movies_crosstab0.index, index=all_movies_crosstab0.index)
    try:
        user_id = int(input('Insert userID'))
    except:
        user_id = 1 # Default user
    
    # Calculating weights
    weights = (user_similarities.query(f"userId!={user_id}")[user_id] / sum(user_similarities.query(f"userId!={user_id}")[user_id]))
    # Exclude user from ratings
    all_movies_crosstab0.loc[user_id,:]==0
    # select movies that the user has not seen
    unseen_movies = all_movies_crosstab0.loc[all_movies_crosstab0.index!=user_id, all_movies_crosstab0.loc[user_id,:]==0]
    # dot product between the unseen movies and the weights
    weighted_averages = pd.DataFrame(unseen_movies.T.dot(weights), columns=["predicted_rating"])
    recommendations = weighted_averages.merge(all_score_df, left_index=True, right_on="movieId")
    recommendations = recommendations.sort_values("predicted_rating", ascending=False).head()
    return recommendations
    
pd.DataFrame(movie_recommendations2_func())


,predicted_rating,movieId,average_rating,no_ratings,score,title,genres
1,2.659011,356,4.164134,329,8.328267,Forrest Gump (1994),Comedy|Drama|Romance|War
2,2.618339,296,4.197068,307,8.113482,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,2.373273,593,4.161290,279,7.690166,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
3,2.348212,2571,4.192446,278,7.734999,"Matrix, The (1999)",Action|Sci-Fi|Thriller
5,2.259498,260,4.231076,251,7.459039,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
